<a href="https://colab.research.google.com/github/YSHebron/rosalind/blob/main/batch/AlphaFold2_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ColabFold v1.5.5: AlphaFold2 w/ MMseqs2 BATCH

<img src="https://raw.githubusercontent.com/sokrypton/ColabFold/main/.github/ColabFold_Marv_Logo_Small.png" height="256" align="right" style="height:256px">

Easy to use AlphaFold2 protein structure [(Jumper et al. 2021)](https://www.nature.com/articles/s41586-021-03819-2) and complex [(Evans et al. 2021)](https://www.biorxiv.org/content/10.1101/2021.10.04.463034v1) prediction using multiple sequence alignments generated through MMseqs2. For details, refer to our manuscript:

[Mirdita M, Schütze K, Moriwaki Y, Heo L, Ovchinnikov S, Steinegger M. ColabFold: Making protein folding accessible to all.
*Nature Methods*, 2022](https://www.nature.com/articles/s41592-022-01488-1)

**Usage**

`input_dir` directory with only fasta files or MSAs stored in Google Drive. MSAs need to be A3M formatted and have an `.a3m` extention. For MSAs MMseqs2 will not be called.

`result_dir` results will be written to the result directory in Google Drive

Old versions: [v1.4](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.4.0/batch/AlphaFold2_batch.ipynb), [v1.5.1](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.5.1/batch/AlphaFold2_batch.ipynb), [v1.5.2](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.5.2/batch/AlphaFold2_batch.ipynb), [v1.5.3-patch](https://colab.research.google.com/github/sokrypton/ColabFold/blob/56c72044c7d51a311ca99b953a71e552fdc042e1/batch/AlphaFold2_batch.ipynb)

<strong>For more details, see <a href="#Instructions">bottom</a> of the notebook and checkout the [ColabFold GitHub](https://github.com/sokrypton/ColabFold). </strong>

-----------

### News
- <b><font color='green'>2023/07/31: The ColabFold MSA server is back to normal. It was using older DB (UniRef30 2202/PDB70 220313) from 27th ~8:30 AM CEST to 31st ~11:10 AM CEST.</font></b>
- <b><font color='green'>2023/06/12: New databases! UniRef30 updated to 2023_02 and PDB to 230517. We now use PDB100 instead of PDB70 (see notes in the [main](https://colabfold.com) notebook).</font></b>
- <b><font color='green'>2023/06/12: We introduced a new default pairing strategy: Previously, for multimer predictions with more than 2 chains, we only pair if all sequences taxonomically match ("complete" pairing). The new default "greedy" strategy pairs any taxonomically matching subsets.</font></b>

In [1]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/drive')
from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

Mounted at /content/drive


In [5]:
#@title Input protein sequence, then hit `Runtime` -> `Run all`

input_dir = '/content/drive/MyDrive/input_fasta' #@param {type:"string"}
result_dir = '/content/drive/MyDrive/output_fasta' #@param {type:"string"}

# number of models to use
#@markdown ---
#@markdown ### Advanced settings
msa_mode = "MMseqs2 (UniRef+Environmental)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
num_models = 5 #@param [1,2,3,4,5] {type:"raw"}
num_recycles = 3 #@param [1,3,6,12,24,48] {type:"raw"}
stop_at_score = 100 #@param {type:"string"}
#@markdown - early stop computing models once score > threshold (avg. plddt for "structures" and ptmscore for "complexes")
use_custom_msa = False
num_relax = 0 #@param [0, 1, 5] {type:"raw"}
use_amber = num_relax > 0
relax_max_iterations = 200 #@param [0,200,2000] {type:"raw"}
use_templates = False #@param {type:"boolean"}
do_not_overwrite_results = True #@param {type:"boolean"}
zip_results = False #@param {type:"boolean"}


In [6]:
#@title Install dependencies
%%bash -s $use_amber $use_templates $python_version

set -e

USE_AMBER=$1
USE_TEMPLATES=$2
PYTHON_VERSION=$3

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  if [ -n "${TPU_NAME}" ]; then
    pip install -q --no-warn-conflicts -U dm-haiku==0.0.10 jax==0.3.25
  fi
  ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold
  ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold
  touch COLABFOLD_READY
fi

# Download params (~1min)
python -m colabfold.download

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    conda config --set auto_update_conda false
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python="${PYTHON_VERSION}" 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.7.0 python="${PYTHON_VERSION}" pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi

In [7]:
#@title Run Prediction

import sys

from colabfold.batch import get_queries, run
from colabfold.download import default_data_dir
from colabfold.utils import setup_logging
from pathlib import Path

# For some reason we need that to get pdbfixer to import
if use_amber and f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
    sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

setup_logging(Path(result_dir).joinpath("log.txt"))

queries, is_complex = get_queries(input_dir)
run(
    queries=queries,
    result_dir=result_dir,
    use_templates=use_templates,
    num_relax=num_relax,
    relax_max_iterations=relax_max_iterations,
    msa_mode=msa_mode,
    model_type="auto",
    num_models=num_models,
    num_recycles=num_recycles,
    model_order=[1, 2, 3, 4, 5],
    is_complex=is_complex,
    data_dir=default_data_dir,
    keep_existing_results=do_not_overwrite_results,
    rank_by="auto",
    pair_mode="unpaired+paired",
    stop_at_score=stop_at_score,
    zip_results=zip_results,
    user_agent="colabfold/google-colab-batch",
)

2024-12-26 16:41:58,201 Running on GPU
2024-12-26 16:41:58,229 Found 5 citations for tools or databases
2024-12-26 16:41:58,231 Query 1/7: 0101 (length 266)


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:02 remaining: 00:00]


2024-12-26 16:42:17,398 Padding length to 276
2024-12-26 16:42:51,092 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=83.2 pTM=0.581
2024-12-26 16:43:14,860 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=85.2 pTM=0.635 tol=3.42
2024-12-26 16:43:39,204 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=85.6 pTM=0.646 tol=1.49
2024-12-26 16:44:03,417 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=85.8 pTM=0.648 tol=1.26
2024-12-26 16:44:03,418 alphafold2_ptm_model_1_seed_000 took 106.0s (3 recycles)
2024-12-26 16:44:27,414 alphafold2_ptm_model_2_seed_000 recycle=0 pLDDT=82.9 pTM=0.543
2024-12-26 16:44:51,544 alphafold2_ptm_model_2_seed_000 recycle=1 pLDDT=84.5 pTM=0.601 tol=2.26
2024-12-26 16:45:15,697 alphafold2_ptm_model_2_seed_000 recycle=2 pLDDT=84.8 pTM=0.618 tol=1.65
2024-12-26 16:45:39,776 alphafold2_ptm_model_2_seed_000 recycle=3 pLDDT=85.1 pTM=0.625 tol=0.708
2024-12-26 16:45:39,777 alphafold2_ptm_model_2_seed_000 took 96.2s (3 recycles)
2024-12-26 16:46:03,998 alphafold2_ptm_mod

COMPLETE: 100%|██████████| 150/150 [elapsed: 00:02 remaining: 00:00]


2024-12-26 16:50:45,617 Padding length to 276
2024-12-26 16:51:09,813 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=83.2 pTM=0.567
2024-12-26 16:51:34,304 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=84.6 pTM=0.616 tol=3.33
2024-12-26 16:51:58,220 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=85.2 pTM=0.639 tol=1.44
2024-12-26 16:52:22,153 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=85.7 pTM=0.644 tol=0.736
2024-12-26 16:52:22,155 alphafold2_ptm_model_1_seed_000 took 96.5s (3 recycles)
2024-12-26 16:52:46,514 alphafold2_ptm_model_2_seed_000 recycle=0 pLDDT=84.4 pTM=0.554
2024-12-26 16:53:10,637 alphafold2_ptm_model_2_seed_000 recycle=1 pLDDT=84.8 pTM=0.595 tol=3.32
2024-12-26 16:53:34,628 alphafold2_ptm_model_2_seed_000 recycle=2 pLDDT=85.2 pTM=0.617 tol=1.82
2024-12-26 16:53:58,772 alphafold2_ptm_model_2_seed_000 recycle=3 pLDDT=85.3 pTM=0.624 tol=3.37
2024-12-26 16:53:58,774 alphafold2_ptm_model_2_seed_000 took 96.5s (3 recycles)
2024-12-26 16:54:23,111 alphafold2_ptm_mode

...

RUNNING:  59%|█████▉    | 89/150 [elapsed: 01:37 remaining: 01:06]

2024-12-26 17:34:39,777 Sleeping for 10s. Reason: RUNNING


COMPLETE: 100%|██████████| 150/150 [elapsed: 01:49 remaining: 00:00]


2024-12-26 17:38:45,739 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=65.3 pTM=0.45
2024-12-26 17:41:40,871 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=69.6 pTM=0.473 tol=21.8
2024-12-26 17:44:35,547 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=70.1 pTM=0.487 tol=5.88
2024-12-26 17:47:30,635 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=70.1 pTM=0.488 tol=3.96
2024-12-26 17:47:30,653 alphafold2_ptm_model_1_seed_000 took 755.2s (3 recycles)
2024-12-26 17:50:25,255 alphafold2_ptm_model_2_seed_000 recycle=0 pLDDT=64.2 pTM=0.427
2024-12-26 17:53:21,210 alphafold2_ptm_model_2_seed_000 recycle=1 pLDDT=67.2 pTM=0.444 tol=17.1
2024-12-26 17:56:15,775 alphafold2_ptm_model_2_seed_000 recycle=2 pLDDT=68.3 pTM=0.453 tol=5.9
2024-12-26 17:59:10,578 alphafold2_ptm_model_2_seed_000 recycle=3 pLDDT=68.8 pTM=0.459 tol=4.26
2024-12-26 17:59:10,591 alphafold2_ptm_model_2_seed_000 took 698.8s (3 recycles)
2024-12-26 18:02:05,510 alphafold2_ptm_model_3_seed_000 recycle=0 pLDDT=67.8 pTM=0.439
202

{'rank': [['rank_001_alphafold2_ptm_model_4_seed_000',
   'rank_002_alphafold2_ptm_model_1_seed_000',
   'rank_003_alphafold2_ptm_model_3_seed_000',
   'rank_004_alphafold2_ptm_model_2_seed_000',
   'rank_005_alphafold2_ptm_model_5_seed_000'],
  ['rank_001_alphafold2_ptm_model_3_seed_000',
   'rank_002_alphafold2_ptm_model_5_seed_000',
   'rank_003_alphafold2_ptm_model_4_seed_000',
   'rank_004_alphafold2_ptm_model_1_seed_000',
   'rank_005_alphafold2_ptm_model_2_seed_000'],
  ['rank_001_alphafold2_ptm_model_4_seed_000',
   'rank_002_alphafold2_ptm_model_3_seed_000',
   'rank_003_alphafold2_ptm_model_5_seed_000',
   'rank_004_alphafold2_ptm_model_2_seed_000',
   'rank_005_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_5_seed_000',
   'rank_002_alphafold2_ptm_model_4_seed_000',
   'rank_003_alphafold2_ptm_model_3_seed_000',
   'rank_004_alphafold2_ptm_model_2_seed_000',
   'rank_005_alphafold2_ptm_model_1_seed_000'],
  ['rank_001_alphafold2_ptm_model_4_seed_000',
 

# Instructions <a name="Instructions"></a>
**Quick start**
1. Upload your single fasta files to a folder in your Google Drive
2. Define path to the fold containing the fasta files (`input_dir`) define an outdir (`output_dir`)
3. Press "Runtime" -> "Run all".

**Result zip file contents**

At the end of the job a all results `jobname.result.zip` will be uploaded to your (`output_dir`) Google Drive. Each zip contains one protein.

1. PDB formatted structures sorted by avg. pIDDT. (unrelaxed and relaxed if `use_amber` is enabled).
2. Plots of the model quality.
3. Plots of the MSA coverage.
4. Parameter log file.
5. A3M formatted input MSA.
6. BibTeX file with citations for all used tools and databases.


**Troubleshooting**
* Check that the runtime type is set to GPU at "Runtime" -> "Change runtime type".
* Try to restart the session "Runtime" -> "Factory reset runtime".
* Check your input sequence.

**Known issues**
* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the structure for a long sequence.
* Google Colab assigns different types of GPUs with varying amount of memory. Some might not have enough memory to predict the structure for a long sequence.
* Your browser can block the pop-up for downloading the result file. You can choose the `save_to_google_drive` option to upload to Google Drive instead or manually download the result file: Click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

**Limitations**
* Computing resources: Our MMseqs2 API can handle ~20-50k requests per day.
* MSAs: MMseqs2 is very precise and sensitive but might find less hits compared to HHblits/HMMer searched against BFD or Mgnify.
* We recommend to additionally use the full [AlphaFold2 pipeline](https://github.com/deepmind/alphafold).

**Description of the plots**
*   **Number of sequences per position** - We want to see at least 30 sequences per position, for best performance, ideally 100 sequences.
*   **Predicted lDDT per position** - model confidence (out of 100) at each position. The higher the better.
*   **Predicted Alignment Error** - For homooligomers, this could be a useful metric to assess how confident the model is about the interface. The lower the better.

**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/sokrypton/ColabFold/issues

**License**

The source code of ColabFold is licensed under [MIT](https://raw.githubusercontent.com/sokrypton/ColabFold/main/LICENSE). Additionally, this notebook uses AlphaFold2 source code and its parameters licensed under [Apache 2.0](https://raw.githubusercontent.com/deepmind/alphafold/main/LICENSE) and  [CC BY 4.0](https://creativecommons.org/licenses/by-sa/4.0/) respectively. Read more about the AlphaFold license [here](https://github.com/deepmind/alphafold).

**Acknowledgments**
- We thank the AlphaFold team for developing an excellent model and open sourcing the software.

- Do-Yoon Kim for creating the ColabFold logo.

- A colab by Sergey Ovchinnikov ([@sokrypton](https://twitter.com/sokrypton)), Milot Mirdita ([@milot_mirdita](https://twitter.com/milot_mirdita)) and Martin Steinegger ([@thesteinegger](https://twitter.com/thesteinegger)).
